# Langchain Agents 
This notebook demonstrates how to create and use agents in Langchain, including the use of tools like DuckDuckGoSearch for real-time information retrieval.

At the end of this notebook, we can see how to create a ReAct agent from a prompt template and use it with a memory system to maintain chat history.



In [20]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq


In [21]:
# load environment variables from .env file
load_dotenv()

True

In [22]:
# Get llms models via the `langchain_groq` package
llm_model = ChatGroq(
    model="llama-3.3-70b-versatile", #llama-3.3-70b-versatile",
    temperature=.5,
    max_tokens=512,
)

## Langchain Tools

The [third-party tools](https://python.langchain.com/docs/integrations/tools/) from the langchain-community ecosytem allow us to extent the capabilities of our agents

In [23]:
# from langchain_community.tools import DuckDuckGoSearchRun
from ddgs import DDGS
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

`DuckDuckGoSearchRun` allows us to perform web searches and retrieve information in real-time, which can be very useful for agents that need to answer questions based on current data.


In [24]:
#api_wrapper = DuckDuckGoSearchAPIWrapper(region="en-en",max_results=10)
#search = DuckDuckGoSearchRun()#,api_wrapper=api_wrapper)

def ddg_search(query: str):
    with DDGS() as ddgs:
        return ddgs.text(query, max_results=5)
search = ddg_search

In [25]:
results = search("Prime Italian Minister")  

In [26]:
results

[{'title': 'Giorgia Meloni - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Giorgia_Meloni',
  'body': 'Giorgia Meloni ( Italian : [ˈdʒordʒa meˈloːni]; born 15 January 1977) is an Italian politician who has served as Prime Minister of Italy since October 2022. She is the first woman to hold the office and the head of the third-longest government in the history of the Italian Republic.'},
 {'title': 'Giorgia Meloni - Italy First Women Prime Minister',
  'href': 'https://www.business-standard.com/about/who-is-giorgia-meloni',
  'body': "Nov 22, 2025 · Giorgia Meloni is Italy 's first female prime minister who created history by leading a right-wing coalition since World War II. Leader of the Fratelli d'Italia (Brothers of Italy party), she has been a member of the Chamber of Deputies from 2006 until 2018."},
 {'title': 'Giorgia Meloni | Party, Policies, & Italy | Britannica',
  'href': 'https://www.britannica.com/biography/Giorgia-Meloni',
  'body': 'Feb 8, 2026 · Giorgia Meloni (b

In [27]:
# Define a tool for the DuckDuckGo search
from langchain.tools import Tool
search_tool = Tool(
    name="DuckDuckGoSearch",
    func=ddg_search,
    description="Use this tool when you need to search for real-time information from DuckDuckGo."
)

In [28]:
from langchain.agents import initialize_agent, AgentType


In [29]:
# Initialize the agent with the DuckDuckGo search tool
agent = initialize_agent(
    tools=[search_tool],
    llm=llm_model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=7,
    agent_name="DuckDuckGoSearchAgent",
    agent_description="An agent that can search the web using DuckDuckGoSearch and summarize findings clearly."
)

# Now we can use the agent to answer questions
results = agent.invoke("Who is the Italian Prime Minister?")  # Example query to the agent



> Entering new AgentExecutor chain...
To find out who the current Italian Prime Minister is, I need to search for the most recent information on the topic. 

Action: DuckDuckGoSearch
Action Input: Italian Prime Minister
Observation: [{'title': 'Italian Prime Minister', 'href': 'https://en.wikipedia.org/wiki/Italian_Prime_Minister', 'body': "The prime minister of Italy, officially the president of the Council of Ministers (Italian: Presidente del Consiglio dei ministri), is the head of government of the Italian Republic. The office of president of the Council of Ministers is established by articles 92–96 of the Constitution of Italy; the president of the Council of Ministers is appointed by the president of the Republic and must have the confidence of the Parliament to stay in office.Prior to the establishment of the Italian Republic, the position was called the President of the Council of Ministers of the Kingdom of Italy (Presidente del Consiglio dei ministri del Regno d'Italia). Fr

### Create a an Agent with DuckDuckGoSearch with Memory
In this section, we will create an agent that can search the web using DuckDuckGoSearch and maintain a memory of previous interactions. 

In [30]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


In [31]:

# Build a chat prompt that includes conversation history so the agent can resolve references.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}"),
])


The react_template instructs the laguage model to:
1. __Think__ about how to solve a user's question
2. __Use tools__ (actions) to gather mor information or perform actions
3. __Observe results__ and update the its thoughts
4. Eventually, reach a __Final Answer__

The following is the explanation of the `react_template` used to create the agent:

```python
react_template = """
'Answer the following questions as best you can. You have access to the following tools:

{tools}
```
+ Introduce the agent and tells it what tools it can use
+ {tools} is a placeholder for the tools that will be used by the agent

```text
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
```

This is the strict __format__ the LLM must follow to:

* Show its reasoning (Thought)

* Choose and execute a tool (Action)

* Feed it input (Action Input)

* Observe results (Observation)

* Loop until it gets to the final answer

```text
Begin!
Question: {input}
Thought:{agent_scratchpad}'
"""
```
+ `{input}` is the actual user question.

+ `{agent_scratchpad}` holds any intermediate steps already completed (e.g., previous Thoughts, Actions, Observations).

+ The agent continues from this point.

In [32]:
# prompt_template defined above with chat history support


In [33]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [34]:
# Simple session store to keep track of chat history per conversation
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [35]:
# Create the agent 
agent = create_react_agent(llm_model, [search_tool], prompt_template)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    handle_parsing_errors=True,
)


In [36]:
# Wrap the agent executor with message history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [37]:
# Now we can use the agent with chat history to answer questions
agent_with_chat_history.invoke(
    {"input": "How many people live in italy?"},
    config={"configurable": {"session_id": "1"}},
)

Parent run 11b0ea0a-055e-42ba-b2b5-332064036b74 not found for run 50540c82-ed7f-4599-8052-30d5aedc6821. Treating as a root run.




> Entering new AgentExecutor chain...
Thought: To find the current population of Italy, I need to search for the most recent data available.

Action: DuckDuckGoSearch
Action Input: Italy population 2024
[{'title': 'Demographics of Italy - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Demographics_of_Italy', 'body': 'Animated population pyramid 1982-2021. Those born during the World wars are marked in dark The demography of Italy is monitored by the Italian National Institute of Statistics (Istat). At the beginning of 2024 , Italy had an estimated population of 58.9 million. [1] Its population density, at 195.7 inhabitants per square kilometre (507/sq mi), is higher than both the EU (106.6/km 2) and ...'}, {'title': 'Italy Population (2026) - Worldometer', 'href': 'https://www.worldometers.info/world-population/italy-population/', 'body': 'Population of Italy : current, historical, and projected population , growth rate, immigration, median age, total fertility rate (TFR), popula

{'input': 'How many people live in italy?',
 'chat_history': [],
 'output': 'The estimated population of Italy in 2024 is approximately 58.9 million.'}

In [41]:
# Let's ask another question to see how the agent uses the chat history
agent_with_chat_history.invoke(
    {"input": "how many € will I have after 3 years if I invest 15900€ with an annual interest rate of 10%?"},
    config={"configurable": {"session_id": "1"}},
)

Parent run 793e92ed-442d-4c2f-8d1e-9085014fd011 not found for run 4937cb2f-594f-4ea6-9b50-9f92ba41f9a9. Treating as a root run.




> Entering new AgentExecutor chain...
To calculate the future value of the investment, we can use the formula:

FV = PV x (1 + r)^n

Where:
FV = Future Value
PV = Present Value (initial investment) = 15900€
r = annual interest rate = 10% = 0.10
n = number of years = 3

FV = 15900 x (1 + 0.10)^3
FV = 15900 x (1.10)^3
FV = 15900 x 1.331
FV = 21153.90

So, after 3 years, you will have approximately 21153.90€.Invalid Format: Missing 'Action:' after 'Thought: To follow the correct format, I should rephrase the answer as follows:

Question: how many € will I have after 3 years if I invest 15900€ with an annual interest rate of 10%?
Thought: To calculate the future value of the investment, we need to use the formula for compound interest.
Action: None
Action Input: NoneNone is not a valid tool, try one of [DuckDuckGoSearch]. Since the calculation is straightforward and does not require real-time data, I can proceed with the calculation directly.
Action: None
Action Input: NoneNone is not a 

{'input': 'how many € will I have after 3 years if I invest 15900€ with an annual interest rate of 10%?',
 'chat_history': [HumanMessage(content='How many people live in italy?'),
  AIMessage(content='The estimated population of Italy in 2024 is approximately 58.9 million.'),
  HumanMessage(content='what is their national anthem called?'),
  AIMessage(content='The national anthem of Italy is called "Il Canto degli Italiani"'),
  HumanMessage(content='which minister was the most famous? a) men and b) women'),
  AIMessage(content='a) men: Giulio Andreotti \nb) women: Emma Bonino'),
  HumanMessage(content='which minister was the most famous? I know it was women and she had second job before in cinema but not build for children'),
  AIMessage(content='Gina Lollobrigida')],
 'output': 'The final answer to the question "how many € will I have after 3 years if I invest 15900€ with an annual interest rate of 10%?" is 21153.90€.'}

### Exercise
Create or Select some tools from the [langchain-community tools](https://python.langchain.com/docs/integrations/tools/) and create a ReAct agent that can use them to solve certain tasks.